### Working With STAC



###### Requirements
- rasterio
- mapboxgl + TOKEN
- requests

`pip install rasterio mapboxgl requests`

In [ ]:
import os
import json
import base64
import requests
import datetime
import itertools
import urllib.parse

from io import BytesIO
from functools import partial
from concurrent import futures
from rasterio.plot import reshape_as_image

from rasterio.features import bounds as featureBounds

from mapboxgl.utils import *
from mapboxgl.viz import *

%pylab inline

token = os.environ["MAPBOX_ACCESS_TOKEN"]

titiler_endpoint = "https://dfu60xgk3j.execute-api.us-east-1.amazonaws.com/"  # Devseed temporary endpoint
stac_endpoint = "https://earth-search.aws.element84.com/v0/search"

#### AOI

In [ ]:
# use geojson.io
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              30.810813903808594,
              29.454247067148533
            ],
            [
              30.88600158691406,
              29.454247067148533
            ],
            [
              30.88600158691406,
              29.51879923863822
            ],
            [
              30.810813903808594,
              29.51879923863822
            ],
            [
              30.810813903808594,
              29.454247067148533
            ]
          ]
        ]
      }
    }
  ]
}

bounds = featureBounds(geojson)

In [ ]:
viz = LinestringViz(
    geojson,
    access_token=token,
    line_width_default=2,
    center=((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2),
    zoom=11,
    style="mapbox://styles/mapbox/satellite-v9?optimize=true"
)
viz.show()

## STAC Search

In [ ]:
date_min="2019-01-01"
date_max="2019-12-11"

start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

query = {
    "collections": ["sentinel-s2-l2a-cogs"],
    "datetime": f"{start}/{end}",
    "query": {
        "eo:cloud_cover": {
            "lt": 5
        }
    },
    "intersects": geojson["features"][0]["geometry"],
    "limit": 1000,
    "fields": {
      'include': ['id', 'properties.datetime', 'properties.eo:cloud_cover'],
      'exclude': ['assets', 'links']
    }
}

headers = {
    "Content-Type": "application/json",
    "Accept-Encoding": "gzip",
    "Accept": "application/geo+json",
}


data = requests.post(stac_endpoint, headers=headers, json=query).json()
print(data["context"])
print()
print(json.dumps(data["features"][0], indent=4))

sceneid = [f["id"] for f in data["features"]]
cloudcover = [f["properties"]["eo:cloud_cover"] for f in data["features"]]
dates = [f["properties"]["datetime"][0:10] for f in data["features"]]

In [ ]:
fig = plt.figure(dpi=100)
fig.autofmt_xdate()
ax = fig.add_subplot(1, 1, 1)
ax.plot(dates, cloudcover, label="Cloud Cover", color="tab:red", linewidth=0.4, linestyle="-.")

ax.legend()

In [ ]:
viz = LinestringViz(
    data,
    access_token=token,
    line_width_default=2,
    center=((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2),
    zoom=11,
    style="mapbox://styles/mapbox/satellite-v9?optimize=true"
)
viz.show()

### DATA Endpoint

https://dfu60xgk3j.execute-api.us-east-1.amazonaws.com/docs#/SpatioTemporal%20Asset%20Catalog

`{endpoint}/stac/tiles/{z}/{x}/{y}.{format}?url={stac_item}&{otherquery params}`


`{endpoint}/stac/crop/{minx},{miny},{maxx},{maxy}.{format}?url={stac_item}&{otherquery params}`


`{endpoint}/stac/point/{minx},{miny}?url={stac_item}&{otherquery params}`


In [ ]:
url_template = "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/{id}"

### Visualize One Item

In [ ]:

r = requests.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = {
        "url": url_template.format(id=sceneid[0]),
        "assets": "B04,B03,B02",  # Simple RGB
        "color_formula": "Gamma RGB 3.5 Saturation 1.7 Sigmoidal RGB 15 0.35",
        "maxzoom": 12,
    }
).json()
print(r)

viz = RasterTilesViz(
    r["tiles"][0] , 
    zoom=8,
    access_token=token,
    tiles_size=256,
    tiles_bounds=r["bounds"],
    center=r["center"],
    tiles_minzoom=r["minzoom"],
    tiles_maxzoom=r["maxzoom"],
)
viz.show()

In [ ]:
r = requests.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = {
        "url": url_template.format(id=sceneid[0]),
        "assets": "B08,B04,B03",  # False Color Infrared
        "color_formula": "Gamma RGB 3.5 Saturation 1.7 Sigmoidal RGB 15 0.35",
        "maxzoom": 14,
    }
).json()
print(r)

viz = RasterTilesViz(
    r["tiles"][0] , 
    zoom=8,
    access_token=token,
    tiles_size=256,
    tiles_bounds=r["bounds"],
    center=r["center"],
    tiles_minzoom=r["minzoom"],
    tiles_maxzoom=r["maxzoom"],
)
viz.show()

In [ ]:
r = requests.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = {
        "url": url_template.format(id=sceneid[0]),
        "expression": "(B08-B04)/(B08+B04)",  # NDVI
        "rescale": "-1,1",
        "maxzoom": 12,
        "color_map": "viridis",
    }
).json()
print(r)

viz = RasterTilesViz(
    r["tiles"][0] , 
    zoom=8,
    access_token=token,
    tiles_size=256,
    tiles_bounds=r["bounds"],
    center=r["center"],
    tiles_minzoom=r["minzoom"],
    tiles_maxzoom=r["maxzoom"],
)
viz.show()

# More

titiler doesn't return only png or jpeg but can also return Numpy array directly

In [ ]:
def fetch_bbox_array(
    sceneid, bbox, assets = None, expression = None, **kwargs,
):
    # STAC ITEM URL
    stac_item = f"https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/{sceneid}"

    xmin, ymin, xmax, ymax = bbox
    
    params = {
        "url": stac_item
    }
    if assets:
        params.update(dict(assets=",".join(assets)))
    elif expression:
        params.update(dict(expression=expression))
    else:
        raise Exception("Missing band or expression input")

    params.update(**kwargs)

    # TITILER ENDPOINT
    url = f"{titiler_endpoint}stac/crop/{xmin},{ymin},{xmax},{ymax}.npy"
    r = requests.get(url, params=params)
    data, mask = numpy.load(BytesIO(r.content), allow_pickle=True)

    return sceneid, data, mask


def _stats(data, mask):
    arr = numpy.ma.array(data)
    arr.mask = mask == 0
    return arr.min().item(), arr.max().item(), arr.mean().item(), arr.std().item()

In [ ]:
_, data, mask = fetch_bbox_array(sceneid[0], bounds, assets=["B02"], max_size=128)

print(data.shape)
print(mask.shape)

imshow(data[0])

In [ ]:
bbox_worker = partial(
    fetch_bbox_array,
    bbox=bounds,
    assets=("B04", "B03", "B02"), 
    color_formula="gamma RGB 3.5, saturation 1.7, sigmoidal RGB 15 0.35",
    max_size=64
)

with futures.ThreadPoolExecutor() as executor:
    results_rgb = list(executor.map(bbox_worker, sceneid))
    
fig = plt.figure(figsize=(10,20))
col = 5
row = math.ceil(len(dates) / col)
for i in range(1, len(results_rgb) + 1):
    fig.add_subplot(row, col, i)
    plt.imshow(reshape_as_image(results_rgb[i-1][1]))

In [ ]:
bbox_worker = partial(
    fetch_bbox_array, bbox=bounds, expression="(B08-B04)/(B08+B04)", max_size=64
)
with futures.ThreadPoolExecutor() as executor:
    results_ndvi = list(executor.map(bbox_worker, sceneid))

In [ ]:
fig = plt.figure(figsize=(10,20))
col = 5
row = math.ceil(len(dates) / col)
for i in range(1, len(results_rgb) + 1):
    fig.add_subplot(row, col, i)
    plt.imshow(results_ndvi[i-1][1][0])

In [ ]:
stats = [_stats(data, mask) for _, data, mask in results_ndvi]

fig, ax1 = plt.subplots(dpi=150)
fig.autofmt_xdate()

ax1.plot(dates, [s[0] for s in stats], label="Min")
ax1.plot(dates, [s[1] for s in stats], label="Max")
ax1.plot(dates, [s[2] for s in stats], label="Mean")
ax1.set_xlabel("Dates")
ax1.set_ylabel("Normalized Difference Vegetation Index")

ax1.legend()